### The Problem

When a trace is done from a node, it will stop prematurely because a valid connection between nodes is missing.

- The START_NODE and END_NODE attributes have been reversed (joined node attirbutes eg. cover/surface level, may be also be transposed).
- The START_NODE and/or END_NODE attributes are incorrect.

### Possible approaches

1. A reversed or missing pipe will, in most cases, create a wrongful outfall/terminus during a downstream trace.
2. A START_NODE, END_NODE value of O_WW or O_CWW is incorrect
3. START_NODE, END_NODE value that has no match in the nodes layer (NODE_ID) is incorrect, or asset owner is not GWW (CWW).
4. The sewer symbol for flow direction is correct more often than pipe feature

### Imports

**trace_gis.trace_sewer** - couple of other ways to import trace_sewer module:
- install from my github repo using magic command: `%pip install https://github.com/timothy_holmes/gww_gis_tools@example`
- copy and paste the contents of trace_sewer.py into your databricks notebook (wrap in another class to follow same references)

**geopandas** - not necessary when pulling data from the curated tables (although shapely might be needed later)

In [1]:
import merge_gis.merge_sewer
import trace_gis.trace_sewer as TS

import geopandas as gpd

### Load Data

Geopandas is hard to install becuase of the dependencies (GDAL), but the important part is that you end up with START_NODE, END_NODE, PIPE_ID columns from the pipes layer.

Can be:
- dataframe with `...` columns (use `g.from_df` method)
- iterable of dicts with `...` keys (use `g.from_dicts` method)
- iterable of tuples with `...` slots (use `g.add_edges `, eg. `functools.reduce(lambda g, t: g.add_edge(*t), iter_tuples, Graph())`)

In [2]:
layer = 'pipes'
pipes_tab_file = merge_gis.merge_sewer.Config.output_template.format(id=layer)
pipes_gdf = gpd.read_file(pipes_tab_file)

### Trace Module API

- Convert dataframe to hash tables to speed up tracing -> Graph object.
- Use Trace object to trace from certain node

In [4]:
g = TS.Graph(TS.DIRECTION.U).from_gdf(pipes_gdf)

tracer = TS.Trace(g)
tr_missing_link = tracer.trace('180058_CWW')
print(tr_missing_link, len(tr_missing_link.nodes))

TraceResult(DIRECTION.U, 180058_CWW) 4


### Simulate corrections in the data

In [5]:
corrections = [
    ('41000_WW', '180060_CWW', 'dummy1'), # WW interface
    ('180058_CWW', '180057_CWW', 'dummy2') # bad node index
]
g_corrected = g
for correction in corrections:
    g_corrected = g_corrected.add_edge(*correction)

tracer_corrected = TS.Trace(g)
tr_corrected = tracer_corrected.trace('180058_CWW')
print(tr_corrected, len(tr_corrected.nodes))

TraceResult(DIRECTION.U, 180058_CWW) 395


### Improvement Metrics

1. number of outfalls (only outfalls should be connections to MW trunks, treatment plants, or ERS)
2. mean length of path for DS traces from every node in network

In [54]:
# inital value for metrics

g_ds = TS.Graph(TS.DIRECTION.D).from_gdf(pipes_gdf)
tracer = TS.Trace(g_ds)
start_nodes = pipes_gdf.START_NODE.unique().tolist()

outfalls_encountered = []
path_lengths = []

for start_node in start_nodes:
    tr = tracer.trace(start_node)
    outfalls_encountered.extend(tr.end_of_path_nodes)
    path_lengths.append(len(tr.nodes) - 1)

print('number of start nodes/traces: ', len(start_nodes))
print('number of encounters with an outfall: ', len(outfalls_encountered))
print('number of unique outfalls: ', len(set(outfalls_encountered))) # metric 1
print('average path length: ', sum(path_lengths) / len(path_lengths)) # metric 2

number of start nodes/traces:  145662
number of encounters with an outfall:  300737
number of unique outfalls:  1646
average path length:  82.76013648034491


In [55]:
# effect of two corrections
corrections = [
    ('41000_WW', '180060_CWW', 'dummy1'), # WW interface
    ('180058_CWW', '180057_CWW', 'dummy2') # bad node index
]

g_corrected = g_ds
for correction in corrections:
    g_corrected = g_corrected.add_edge(*correction)

tracer = TS.Trace(g_corrected)
start_nodes = set(pipes_gdf.START_NODE.tolist())

outfalls_encountered = []
path_lengths = []

for start_node in start_nodes: # O(n^2*log(n))
    tr = tracer.trace(start_node)
    outfalls_encountered.extend(tr.end_of_path_nodes)
    path_lengths.append(len(tr.nodes) - 1)

print('number of start nodes/traces: ', len(start_nodes))
print('number of encounters with an outfall: ', len(outfalls_encountered))
print('number of unique outfalls: ', len(set(outfalls_encountered))) # metric 1
print('average path length: ', sum(path_lengths) / len(path_lengths)) # metric 2

number of start nodes/traces:  145662
number of encounters with an outfall:  301520
number of unique outfalls:  1644
average path length:  83.4732531476981


### Identify legit outfalls

`outfalls` ~ possible erroneous outfall

Apparent outfalls can arise from:
- reversed pipe (edge), ie. `START_NODE, END_NODE` are transposed
- legitimate outfalls, eg. emergency relief (overflow) structure (ERS), treatment plant outfall (eg. ATL1P), treatment plant discharge point (eg. WEO1)
- missing other authority node

In [32]:
# get nodes for node reference (again no need for geometry yet)
nodes_tab_file = merge_gis.merge_sewer.Config.output_template.format(id='nodes')
nodes_gdf = gpd.read_file(nodes_tab_file)

In [60]:
from collections import Counter

count_outfalls = Counter(outfalls_encountered)
outfalls = sorted(set(outfalls_encountered), key=count_outfalls.get, reverse=True)

print(len(outfalls))

1644


In [67]:
assessed = {
    'good': [
        '38264_CWW' # 'ALT1P' ATP inlet
        '44177_CWW' # 'WEO1P' ATP discharge point
    ],
    'bad': {
        # node_ids: (action to fix)
    }
}

outfalls = filter(lambda o: (o not in assessed['good']) and (o not in assessed['bad']), outfalls)
outfalls_encountered = filter(lambda o: (o not in assessed['good']) and (o not in assessed['bad']), outfalls)

**Is this an ERS?** 

This only applies to CWW. I don't think WW ERS nodes are marked as such.

In [68]:

outfalls_where = (nodes_gdf.NODE_ID.isin(outfalls))
outfalls_view = nodes_gdf[outfalls_where]
ers_where = (nodes_gdf[outfalls_where].NODE_REF.str.startswith('ERS'))
ers_view = outfalls_view[ers_where]
confirmed_ers_where = (
    # in most scenarios, 3 of 3 will be true
    (ers_view.NODE_TYPE == 'AP_ENDOFPIPE') | # node type is 'AP_ENDOFPIPE', or
    (ers_view.NODE_REF.str.endswith('P')) | # node type ends with 'P', or
    (ers_view.NODE_ID.apply(lambda x: len(tuple(v for k, v in g.nodes.items() if x in v)) == 1)) # node has exactly one inlet (neighbour)
)
confirmed_ers_view = ers_view[confirmed_ers_where]
confirmed_ers_nodes = confirmed_ers_view.NODE_ID.tolist()
confirmed_not_ers_nodes = ers_view[~ers_view.NODE_ID.isin(confirmed_ers_nodes)].NODE_ID.tolist()

assessed['good'].extend(confirmed_ers_nodes)
assessed['bad'].update({o: 'reverse only upstream pipe' for o in confirmed_not_ers_nodes})

In [63]:
outfalls = filter(lambda o: (o not in assessed['good']) and (o not in assessed['bad']), outfalls)

**Is this an end of pipe node, at the top of the network?** 

End of pipe nodes should have only 1 connected pipe. Unfortunately, other legit outfalls might the same criteria:
- ERS (not detected in previous step)
- treatment plant outfalls (need to confirm none are marked end of pipe; no way to identify from data)
- need to see there is WW equivalent to CWW's 'AP_ENDOFPIPE'

In [65]:
outfalls_where = (nodes_gdf.NODE_ID.isin(outfalls))
outfalls_view = nodes_gdf[outfalls_where]
end_of_pipe_where = (
    # end of pipe should always be a single inlet; except at a pump station, it may be a mislabelled ERS
    (outfalls_view.NODE_TYPE == 'AP_ENDOFPIPE') & 
    ~outfalls_view.NODE_REF.str.startswith('SPS') &
    outfalls_view.NODE_ID.apply(lambda x: len(tuple(v for k, v in g.nodes.items() if x in v)) == 1)
) 
end_of_pipe_view = outfalls_view[end_of_pipe_where]

# assessed['bad'].update({o: 'reverse only upstream pipe' for o in confirmed_not_ers_nodes})